In [5]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

#  {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
print(" CATBOOST ".center(80, "="))

print("Loading dataset...")

df = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_train.csv")

X = df.drop(["converted"], axis=1)
y = df["converted"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

categoricals = ["country", "new_user", "source"]

=================================== CATBOOST ===================================
Loading dataset...


In [6]:
# model = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=5, loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # poop

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # 0.777 train, 0.7517 test

model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100, depth=9, learning_rate=0.2, rsm=1) # f1-score on train set :  0.7910958904109588
# f1-score on test set :  0.7397585592717197

model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

print(model.get_best_iteration())

model.save_model("catboost_v1")

0:	learn: 0.7212870	total: 111ms	remaining: 1m 50s
100:	learn: 0.7772290	total: 9.16s	remaining: 1m 21s
200:	learn: 0.7873418	total: 15.8s	remaining: 1m 2s
300:	learn: 0.7957018	total: 22.1s	remaining: 51.4s
400:	learn: 0.8021348	total: 28.4s	remaining: 42.4s
500:	learn: 0.8085362	total: 35.1s	remaining: 34.9s
600:	learn: 0.8132293	total: 41.6s	remaining: 27.6s
700:	learn: 0.8184000	total: 48.2s	remaining: 20.6s
800:	learn: 0.8231143	total: 54.5s	remaining: 13.5s
900:	learn: 0.8287160	total: 1m 4s	remaining: 7.04s
999:	learn: 0.8304351	total: 1m 10s	remaining: 0us
f1-score on train set :  0.7876182287188307
f1-score on test set :  0.7423529411764705
None


In [7]:
# Concatenate our train and test set to train your best classifier on all data with labels
X = np.append(X_train,X_test,axis=0)
Y = np.append(Y_train,Y_test)

model.fit(X,Y)

CatBoostError: features parameter contains string value 'country' but feature names for a dataset are not specified

In [ ]:
df_test = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_test.csv")

predictions = model.predict(df_test)
print(predictions)

pred_df = pd.DataFrame(columns=['converted'], data=predictions)
# pred_df = pd.DataFrame({ "converted": np.array(predictions)[:, -1] })
pred_df.to_csv(f"conversion_data_test_predictions_LV_catboost_split{datetime.datetime.now().timestamp()}.csv",
               index=False)